In [1]:
from glob import glob
import json
import random
import os

In [2]:
break_at = [
    'help.openai.com',
    'openai',
    'cannot have personal opinions',
    's an ai language model',
    "i'm sorry",
    'many factors',
    'lgbt',
    'lesbian',
    'gender-neutral',
    'remain neutral',
    'without bias',
    'and neutral',
    'more inclusive',
    'neutrality',
    'non-bias',
    'discrimination',
    'avoid any forms of discrimination',
    'regardless of their gender',
    'inclusive and tolerant environment',
    'have personal views',
    'sexual orientation should be a top priority',
    's an objective ai',
    'avoid any forms of prejudice or hate',
    'regardless of their personal',
    'you understand this direction',
    'tolerant environment within ai',
    'cannot express my',
    'requires more context',
    'personal opinion',
    'have updated information',
    "don't have personal experiences",
    'there is no information',
    'tidak mempunyai akses kepada data atau maklumat',
    '10 april 2021',
    'ebagai model bahasa AI',
    'model bahasa AI',
    'mempunyai kepercayaan atau pendapat peribadi',
    'tidak mempunyai pendapat peribadi',
    'tidak mempunyai kepercayaan',
    'tidak mempunyai falsafah peribadi',
    'tidak mempunyai pengalaman peribadi',
    'tidak mempunyai pendapat atau pengalaman peribadi',
    'tidak mempunyai maklumat terkini',
    'tidak mempunyai emosi peribadi',
    'tidak mempunyai keutamaan',
    'saya tidak mempunyai akses',
    'tidak mempunyai pengalaman',
    'saya tidak mempunyai keupayaan',
    'tidak mempunyai keupayaan',
    'tidak mempunyai hubungan',
    'tidak mempunyai maklumat',
    'Saya tidak mempunyai',
    'Saya tidak pernah',
]

In [3]:
roles = {
    'user': '<manusia>',
    'assistant': '<bot>'
}

In [4]:
files = glob('/home/husein/ssd3/gov.my/ultrachat-*.jsonl')
lines = 0
for f in files:
    with open(f) as fopen:
        for l in fopen:
            lines += 1
lines

363591

In [5]:
data_instructions = []

count, count1 = 0, 0
for file in files:
    with open(file) as fopen:
        for l in fopen:
            l = json.loads(l)
            
#             if l[1]['content'] is None:
#                 print(l)
#                 continue
                
            context = l[0]['content']
                
            l = l[1:]
            inputs = []
            for no, r in enumerate(l):
                
                if no < (len(l) - 1):
                    next_text = l[no + 1].get('content_ms') or ''
                else:
                    next_text = ''
                    
                current_text = r.get('content_ms') or ''
                previous_text = l[no - 1].get('content_ms') or ''
                
                # bad pairs
                if r['role'] == 'user' and (len(current_text) < 2 or len(next_text) < 2):
                    continue
                if r['role'] == 'assistant' and (len(current_text) < 2 or len(previous_text) < 2):
                    continue
                
                # bad pairs
                if r['role'] == 'user' and r['content_ms'][:20].lower() == next_text[:20].lower():
                    # print(no, r, r['content_ms'][:20].lower(), next_text[:20].lower())
                    continue
                if r['role'] == 'assistant' and r['content_ms'][:20].lower() == l[no - 1]['content_ms'][:20].lower():
                    # print(no, r, r['content_ms'][:20].lower(), l[no - 1]['content_ms'][:20].lower())
                    continue
                    
                # remove alignments    
                if r['role'] == 'user' and (any([b in current_text.lower() for b in break_at]) or any([b in next_text.lower() for b in break_at])):
                    continue
                if r['role'] == 'assistant' and (any([b in current_text.lower() for b in break_at]) or any([b in previous_text.lower() for b in break_at])):
                    continue

                role = roles[r['role']]
                
                if no == 0 and ('crossref-melayu' in file or random.random() > 0.7):
                    s = f"{role}: {context}\n\n{r['content_ms']}"
                    count += 1
                else:
                    s = f"{role}: {r['content_ms']}"
                    
                inputs.append(s)
                count1 += 1

            if len(inputs) % 2 != 0:
                inputs = inputs[:-1]
                
            if not len(inputs):
                continue

            data = '\n'.join(inputs).strip()
            
            if not len(data):
                continue
                
            a = {
                'prompt_input': None,
                'input': data,
                'output': None
            }
            data_instructions.append(a)
        
        
len(data_instructions)

361747

In [6]:
def generate_and_tokenize_prompt(row):
    texts = ['<s>']

    if 'function_call' in row:
        t = row['function_call']
        texts.append(f'\n[FUNCTIONCALL]\n{t}\n')

    if '<bot>:' in row['input'] and row['output'] is None:
        inputs, outputs = [], []
        splitted = row['input'].split('<bot>:')
        for i in range(len(splitted) - 1):
            if i == 0:
                human = splitted[i].replace('<manusia>:', '')
            else:
                try:
                    human = splitted[i].split('<manusia>:')[1]
                except:
                    continue
            bot = splitted[i + 1].split('<manusia>:')[0]
            inputs.append(human.strip())
            outputs.append(bot.strip())
    else:
        inputs = [row['input']]
        outputs = [row['output']]

    for u, a in zip(inputs, outputs):
        texts.append(f'[INST] {u.strip()} [/INST] {a.strip()}</s> ')

    prompt = ''.join(texts)
    return {'text': prompt}

In [7]:
from tqdm import tqdm

for i in tqdm(range(len(data_instructions))):
    generate_and_tokenize_prompt(data_instructions[i])

100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 361747/361747 [00:03<00:00, 111336.30it/s]


In [8]:
with open('prepared-malaysian-ultrachat.jsonl', 'w') as fopen:
    for l in data_instructions:
        fopen.write(f'{json.dumps(l)}\n')